In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ChatVectorDBChain
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredFileLoader
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["QDRANT_API_KEY"] = os.getenv("QDRANT_API_KEY")
os.environ["QDRANT_HOST"] = os.getenv("QDRANT_HOST")

In [ ]:
where_the_dir_is = r'D:\Downloaded\BibleBuddy Docs\NetBible\Test'
# where_the_dbs_will_be_saved = r'D:\Downloaded\BibleBuddy\db\Bibles\Collections\KJV'

In [ ]:
# loader_cls = lambda path: UnstructuredFileLoader(path)
# This will allow whole directorys to be embedded. 
loader = DirectoryLoader(where_the_dir_is)
# loader_cls = lambda path: UnstructuredFileLoader(path, mode="elements", max_tokens=2000)

# loader.loader_cls = lambda path: UnstructuredFileLoader(path, mode="elements")
docs = loader.load()
len(docs)

In [2]:
from langchain.document_loaders import UnstructuredEPubLoader
loader = UnstructuredEPubLoader(r"D:\Downloaded\BibleBuddy Docs\NetBible\NET21NOTELESS.epub")
docs = loader.load()

In [3]:
documents = loader.load()

In [4]:
# from langchain.document_loaders import TextLoader
# loader = TextLoader('db/paul_graham_essay.txt')
# documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1060, which is longer than the specified 1000
Created a chunk of size 1053, which is longer than the specified 1000
Created a chunk of size 1153, which is longer than the specified 1000
Created a chunk of size 1075, which is longer than the specified 1000
Created a chunk of size 1179, which is longer than the specified 1000
Created a chunk of size 1107, which is longer than the specified 1000
Created a chunk of size 1018, which is longer than the specified 1000
Created a chunk of size 1011, which is longer than the specified 1000
Created a chunk of size 1087, which is longer than the specified 1000


In [5]:
# host = "biblebuddycluster"
api_key = os.environ["QDRANT_API_KEY"]
hosturl = os.environ["QDRANT_HOST"]
collectionname = 'netbible'
qdrant = Qdrant.from_documents(docs, embeddings, url=hosturl, prefer_grpc=True, api_key=api_key, collection_name=collectionname)


In [6]:
query = "What did Jesus say about Sabbath?"
docs = qdrant.similarity_search(query)
docs[0]

Document(page_content='2:23 Jesus was going through the grain\r\nfields on a Sabbath, and his disciples began to pick some heads of wheat\r\nas they made their way.\n\n2:24 So the Pharisees\r\nsaid to him, “Look, why are they doing what is against the law on the\r\nSabbath?”\n\n2:25 He said to them, “Have you\r\nnever read what David did when he was in need and he and his companions\r\nwere hungry—\n\n2:26 how he entered the house of\r\nGod when Abiathar was high priest and ate the sacred bread, which is\r\nagainst the law for any but the priests to eat, and also gave it to his\r\ncompanions?”\n\n2:27 Then he said to them, “The\r\nSabbath was made for people, not people for the Sabbath.\n\n2:28 For this reason the Son of Man is lord even of\r\nthe Sabbath.”\n\nMark\r\nChapter 3\n\nHealing a Withered Hand\n\n3:1 Then Jesus entered the synagogue\r\nagain, and a man was there who had a withered hand.\n\n3:2 They watched Jesus closely to see if he would\r\nheal him on the Sabbath, so that 

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
system_template="""Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. Provide a scripture reference.
ALWAYS return a "sources" part in your answer.
The "sources" part should be a reference to the source of the document from which you got your answer.

The output should be a markdown code snippet formatted in the following schema:
```json
{{
answer: is foo
sources: xyz
}}
```
Begin!
----------------
{context}"""
messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]
prompt = ChatPromptTemplate.from_messages(messages)

In [ ]:
qa = ChatVectorDBChain.from_llm(ChatOpenAI(temperature=0), qdrant,qa_prompt=prompt, return_source_documents=True)

In [ ]:
chat_history = []
query = "What is Exodus about?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

In [ ]:
result['source_documents']

Testing persistance

In [ ]:
!pip install qdrant_client

In [ ]:
# from langchain import Qdrant
from qdrant_client import QdrantClient
from langchain.vectorstores import Qdrant

host = "biblebuddycluster"
api_key = os.environ["QDRANT_API_KEY"]
hosturl = os.environ["QDRANT_HOST"]
client = QdrantClient(url=hosturl, prefer_grpc=True, api_key=api_key )
collection_name = "KJV"
embeddings = OpenAIEmbeddings().
qdrant2 = Qdrant(client, collection_name, embedding_function=embeddings.embed_query)

In [ ]:
qdrant2

In [ ]:
query = "What did Jesus say about Sabbath?"
docs = qdrant2.similarity_search(query)
docs[0]